# idact - Prometheus sandbox

## Initial setup

Add `idact` to path:

In [ ]:
import sys
import os
import bitmath
import getpass
import contextlib
import fabric
import logging
from pprint import pprint

def append_idact_path():
    idact_path = os.path.realpath(os.path.join(os.getcwd(), '../'))
    sys.path.append(idact_path)
append_idact_path()

from idact import *
from idact.detail.auth.set_password import set_password

os.environ['IDACT_KEY_LOCATION'] = os.path.join(os.getcwd(), '../.notebook-ssh')
try:
    os.mkdir(os.environ['IDACT_KEY_LOCATION'])
except OSError as e:
    print(e)

USER = 'plggarstka'

Hide debug information, setup context manager stack (for testing purposes)

## Add cluster (only first run)

In [2]:
cluster = add_cluster(name="pro",
                      user=USER,
                      host="pro.cyfronet.pl",
                      port=22,
                      auth=AuthMethod.PUBLIC_KEY,
                      key=KeyType.RSA,
                      install_key=True)

2018-07-29 22:58:21 INFO: Generating public-private key pair.


## Load cluster (subsequent runs)

In [3]:
load_environment('.idact-env')
cluster = show_cluster("pro")
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_qj', install_key=False, disable_sshd=False)

In [4]:
set_log_level(logging.INFO)
#set_log_level(logging.DEBUG)

In [5]:
node = cluster.get_access_node()
node

Node(pro.cyfronet.pl:22, None)

In [6]:
node.run('whoami')

'plggarstka'

In [7]:
save_environment('.idact-env')

In [8]:
node.run('whoami')

'plggarstka'

In [9]:
node.run('hostname')

'login01.pro.cyfronet.pl'

## Allocate nodes

In [10]:
nodes = cluster.allocate_nodes(nodes=2,
                               cores=1,
                               memory_per_node=bitmath.MiB(100),
                               walltime=Walltime(minutes=20),
                               native_args={
                                   '--partition': 'plgrid-testing',
                                   '--account': 'ccbmc7'
                               })

In [11]:
nodes

Nodes([Node(NotAllocated),Node(NotAllocated)])

In [12]:
nodes.wait()
nodes

Nodes([Node(p0623:58365, 2018-07-29 21:21:08.491517+00:00),Node(p0624:55559, 2018-07-29 21:21:08.491517+00:00)])

## Run commands

In [13]:
nodes[0].run('whoami')

'plggarstka'

In [14]:
nodes[0].run('hostname')

'p0623'

In [15]:
nodes[1].run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          12362692 plgrid-te     wrap plggarst  R       0:12      2 p[0623-0624]'

In [16]:
nodes[1].run('hostname')

'p0624'

## Tunnel

In [17]:
tunnel = nodes[0].tunnel(here=9000, there=10000)

In [18]:
tunnel

MultiHopTunnel(9000:10000)

In [19]:
tunnel.close()

## Close

In [20]:
nodes.running()

True

In [21]:
nodes.cancel()

In [23]:
nodes.running()

False

In [24]:
node.run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)'